파이썬 컬렉션 처리 함술ㄹ 함수형 프로그래밍의 관점에서 살펴본다.

반복 가능 객체나 시퀀스를 재귀나 명시적 루프를 사용해 처리ㅏ기 위한 추가 디자인 패턴을 살펴본다. 그리고 제너레이터 식에서 데이터의 컬렉션에 대해 scalar() 함수를 적용하는 방법을 살펴본다.

### 함수의 다양성에 대한 정리

함수를 두 가지 넓은 부류로 구별해야 한다.
* 스칼라 함수는 개별 값에 적용할 수 있고, 개별적인 결과를 내놓는다. abs(), pow() 등의 함수나 math 모듈에 있는 모든 함수들이 스칼라 함수의 예다.

* Collection 함수는 반복 가능한 컬렉션에 작용한다. 

컬렉션 함수는 다음과 같이 세 가지로 나눌 수 있다.

* reduction: 컬렉션에 있는 여러 값을 함수를 사용해 겹쳐 호출함으로써 최종적으로 단일 값을 만들어 낸다. aggregate 함수라고도 부른다. 
* mapping: 함수를 컬렉션의 모든 원소에 적용한다. 결과는 입력 컬렉션과 크기가 같은 다른 컬렉션이다.
* filter: 컬렉션의 모든 원소에 함수를 적용하여 빌부 원소는 버리고 일부 원소를 통과시키는 것이다. 

### 반복 가능 객체로 작업하기

보통 for 루프를 사용해 컬렉션에 대한 작업을 수행한다. 튜플, 리스트, 맵, 집합 등의 실체화한 컬렉션에 대해 작업하는 경우, for 루프는 상태를 명시적으로 관리한다. 이는 순수한 함수형 프로그래밍으로부터 벗어난 것이지만, 파이썬에 필요한 최적화를 반영하는 것이기도 하다. for문을 평가하는 과정에서 생기는 반복자 객체에만 상태가 한정되도록 보장할 수 있다면, 순수한 함수형 프로그래밍에서 크게 벗어나지 않은 상태에서 for문을 활용할 수 있을 것이다. 예를 들어 for 루프의 변수를 그 루프에 대해 들여쓴 본문의 바깥에서 사용할 경우 순수한 함수형 프로그래밍에서 상당히 멀리 벗어난 것이다.

for 루프를 사용하는 일반적인 응용으로는 풀기(처리(감싸기(반복 가능객체))) 디자인 패턴을 들 수 있다. 감싸기() 함수는 반복 가능 객체의 각 원소를 정렬하기 위한 키나 다른 값과 함께 2-튜플로 묶는다. 그 후 이 2-튜플을 바탕으로 처리를 수행한다. 마지막으로, 풀기() 함수는 감싸기 위해 사용했던 키 값을 버리고, 원래의 원소를 복원한다.

이러한 작업은 함수형 문맥에서 매우 자주 잇어나는 일이기 때문에 이러한 경우에 자주 사용하는 두 함수가 존재한다.

In [1]:
fst = lambda x: x[0]
snd = lambda x: x[1]

두 가지 모두 처리() 나 풀기() 함수에서 편리하게 사용할 수 있다.

다른 일반적인 패턴은 감싸기(감싸기(감싸기()))다. 이경우, 간단한 튜플에서 시작하여 다른 결과를 추가한 후 좀 더 크고 복잡한 튜플을 만든다. 이러한 패턴을 변경한 것 중 흔히 사용하는 것으로는 확장(확장(확장()))이 있다. 여기서는 원래의 튜플을 감싸는 대신 한 번 확장할 때마다 새로운 namedtuple 인스턴스를 만들어낸다. 이 두 가지 모두 첨가 디자인 패턴이라 부를 수 있다.

첨가 디자인 패턴을 위도와 경도 값을 다루기 위해 사용할 수 있다. 첫 단계는 어떤 경로에 속한 (lat, lon) 이라는 단순한 지점 구간을 나타내는 (begin, end) 쌍으로 만드는 것이다. 각 쌍은((lat, lon) (lat, lon))가 될 것이다.

다음 절에서 파일의 내용에 대한 제너레이터 함수를 만드는 방법을 볼 것이다. 이 반복 가능 객체에는 우리가 처리할 입력 데이터가 들어갈 것이다.

데이터를 준비하고 나면, 그 이후의 절에서는 각 구간에 대한 하버사인 거리를 구하도록 구간을 감싸는 방법을 살펴본다. 이렇게 감싸기(감싸기(감싸기())) 패턴으로 처리한 마지막 결과는 3-튜플((lat-lon), (lat-lon), distance)의 시퀀스다. 그 후 이를 분석하여 최장거리나 최단거리, 어떤 경로를 둘러싼 직사각형 또는 데이터를 요약한 다른 값 등을 구할 수 있다.

### XML 파일 구문 분석하기

XML[확장 가능한 마크업 언어] 파일을 구문 분석하여 위경도 쌍을 가져오는 것부터 시작할 것이다. 이를 통해 파이썬에서 확실히 팜수형이 아닌 기능을 감싸 값의 반복 가능한 시퀀스를 만드는 과정을 보여줄 것이다. xml.etree 모듈을 사용할 것이고, 구문 분석한 결과인 ElementTree 객체에는 모든 값에 대해 방문할 수 있는 findall() 메서드가 들어 있다.

다음과 같은 XML 엘리먼트를 찾을 것이다.